# `requests`

## HTTP  

Hyper-text Transfer Protocol е протокол за трансфер на информация (в приложния слой на [OSI](https://www.youtube.com/watch?v=mRuSoU_Pw4o) модела), който е стандарт за комуникация в мрежата.

Версии:
* HTTP/1 - от 1996г.
* HTTP/2 - от 2015г.
* HTTP/3 - от 2022г.

Примерна HTTP заявка:

```http
GET / HTTP/1.1
Host: www.example.com
User-Agent: Mozilla/5.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: en-GB,en;q=0.5
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
```

Винаги първия ред е във формат `{HTTP метода} {URI} HTTP/{версия}`. В случая искаме да вземем ресурсът, намиращ се на `/` (т.е. root-a на example.com) с GET метода.

След него на всеки ред стои 1 хедър (заглавие) във формат `{име}: {стойност}`

Примерен HTTP отговор на заявка:

```http
HTTP/1.1 200 OK
Date: Mon, 23 May 2005 22:38:34 GMT
Content-Type: text/html; charset=UTF-8
Content-Length: 155
Last-Modified: Wed, 08 Jan 2003 23:11:55 GMT
Server: Apache/1.3.3.7 (Unix) (Red-Hat/Linux)
ETag: "3f80f-1b6-3e1cb03b"
Accept-Ranges: bytes
Connection: close

<html>
  <head>
    <title>An Example Page</title>
  </head>
  <body>
    <p>Hello World, this is a very simple HTML document.</p>
  </body>
</html>
```

Пърият ред винаги е във формат `HTTP/{версия} {статус код} {има на статус кода}`. В случая той е "200 OK". (лист с всички кодове: [тук](https://http.cat))

След това са хедърите, след които има празен ред и започва тялото на отговора. В случая това е HTML страница, "намираща" се на example.com.

### Методи

* GET (взимане на ресурс) (заявката *може да няма* тяло)
* POST (изпращане на ресурс) (заявката *има* тяло)
* PUT (заменяне на ресурс) (заявката *има* тяло)
* PATCH (промяна на част от ресурс) (заявката *има* тяло)
* DELETE (изтриване на ресурс) (заявката *може да няма* тяло)
* HEAD (взимане само на хедърите на ресурс (като GET, но без върнато тяло))
* OPTIONS (взимане на методите, поддържани от ресурса) (заявката *може да няма* тяло)

###  Статус кодове

* 200 - 299: Успешно изпълнена заявка
* 300 - 399: Пренасочване
* 400 - 499: Грешка на клиента
* 500 - 599: Грешка на сървъра

[лист с всички](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)


## Добавяне на `requests`

Библиотеката `requests` не е вградена в езика (repo: https://github.com/psf/requests), затова трябва да се инсталира допълнително. 

Чрез package manager-a [`pip`](https://pip.pypa.io/en/stable/installation/) това става с командата:

```bash
pip install requests  # sometimes pip3 is the right one though
```

In [2]:
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.2 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 2.6 MB/s eta 0:00:004.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.0 MB/s eta 0:00:00 MB/s eta 0:00:01


След това трябва да се включи в текущия модул чрез `import requests`:

In [3]:
import requests

### Методи

В модула има по една функция за всеки един HTTP метод. Всички те са с еднаква сигнатура, която по подразбиране изисква само един аргумент - URL адреса на ресурса.

In [16]:
requests.get("https://fmi.uni-sofia.bg")

<Response [200]>

In [11]:
requests.post("https://google.com")

<Response [405]>

In [14]:
requests.delete("https://tesla.com")  # cancel Musk

<Response [501]>

Всеки един от тези методи връщат `Response` обект, в който се съдържа целия отговор.

### Проверка дали заявката е успешна

Статус кода на отговора можем да вземем чрез `status_code` атрибута (тип `int`):

In [28]:
for url in [
    "https://httpbin.org/status/204",
    "https://httpbin.org/status/404",
]:
    response = requests.get(url)

    if response.status_code in range(200, 400):
        print("Request went well. Status code = ", response.status_code)
    else:
        print("Sum Ting Went Wong. Status code = ", response.status_code)

Request went well. Status code =  204
Sum Ting Went Wong. Status code =  404


Горния начин на проверка обаче е често срещан - всички 2хх и 3хх статус кодове означават, че грешка на клиента или съвръра не е имало. Затова `Response` обекта има предефиниран `__bool__`, който оценява `self` по същия начин като в горното сравнение:

In [34]:
for url in [
    "https://httpbin.org/status/204",
    "https://httpbin.org/status/404",
]:
    response = requests.get(url)

    if response:
        print("Request went well. Status code = ", response.status_code)
    else:
        print("Sum Ting Went Wong. Status code = ", response.status_code)

Request went well. Status code =  204
Sum Ting Went Wong. Status code =  404


Ако пък искаме да работим с изключения, може да извикаме `raise_for_status`, което би хвърлило `HTTPError` ако не е успешна заявката:

In [37]:
from requests.exceptions import HTTPError

for url in [
    "https://httpbin.org/status/204",
    "https://httpbin.org/status/404",
    "example.com",  # thisis invalid so it will not even get to the `.raise_for_status()`
]:
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')

    except Exception as err:
        print(f'Other error occurred: {err}')
    
    else:
        print('Success!')

Success!
HTTP error occurred: 404 Client Error: NOT FOUND for url: https://httpbin.org/status/404
Other error occurred: Invalid URL 'example.com': No scheme supplied. Perhaps you meant http://example.com?


### Взимане на хедърите и съдържанието от отговора

In [42]:
response = requests.get("https://httpbin.org/json")
response

<Response [200]>

Заглавните части (хедърите) са в `headers`, което връща обект, подобен на `dict[str, str]`, но с case-insensitive ключове:

In [55]:
response.headers

{'Date': 'Sun, 27 Nov 2022 23:22:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '429', 'Connection': 'keep-alive', 'Server': 'gunicorn/19.9.0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true'}

In [57]:
response.headers["CONTENT-TYPE"]

'application/json'

`content` връща тялото като `bytes`:

In [51]:
response.content

b'{\n  "slideshow": {\n    "author": "Yours Truly", \n    "date": "date of publication", \n    "slides": [\n      {\n        "title": "Wake up to WonderWidgets!", \n        "type": "all"\n      }, \n      {\n        "items": [\n          "Why <em>WonderWidgets</em> are great", \n          "Who <em>buys</em> WonderWidgets"\n        ], \n        "title": "Overview", \n        "type": "all"\n      }\n    ], \n    "title": "Sample Slide Show"\n  }\n}\n'

Ако очакваме да е текстово съдържанието, можем да ползваме и `text`, за да ги конвертираме в `str`. По подразбиране `encoding`-ът е "utf-8", може да се променя от едноименния атрибут.

In [53]:
print(response.text)  # print will just prettify the output

{
  "slideshow": {
    "author": "Yours Truly", 
    "date": "date of publication", 
    "slides": [
      {
        "title": "Wake up to WonderWidgets!", 
        "type": "all"
      }, 
      {
        "items": [
          "Why <em>WonderWidgets</em> are great", 
          "Who <em>buys</em> WonderWidgets"
        ], 
        "title": "Overview", 
        "type": "all"
      }
    ], 
    "title": "Sample Slide Show"
  }
}



JSON като формат за пренос на данни е най широко-използвания сред HTTP услугите поради леснотата на работата с него. В случая отговора е точно в такъв вид и можем да използваме и помощния метод `json()`, който ни връща отговора като `dict` (или `list`, е зависимост от обекта, който седи като корен на JSON-a):

In [48]:
response.json()

{'slideshow': {'author': 'Yours Truly',
  'date': 'date of publication',
  'slides': [{'title': 'Wake up to WonderWidgets!', 'type': 'all'},
   {'items': ['Why <em>WonderWidgets</em> are great',
     'Who <em>buys</em> WonderWidgets'],
    'title': 'Overview',
    'type': 'all'}],
  'title': 'Sample Slide Show'}}